### **Mono Audio Feature Extraction**

In [1]:
from __future__ import print_function
import librosa
import librosa.display
import numpy as np
import pandas as pd
import os

In [2]:
file_path = 'Mono_Audio/Fall_Mono'
output_path = 'Mono_Audio_Feature/Fall_Feature'

fall_list = os.listdir(file_path)
count = 1

for file_name in fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'F{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

In [3]:
file_path = 'Mono_Audio/Non_Fall_Mono'
output_path = 'Mono_Audio_Feature/Non_Fall_Feature'

fall_list = os.listdir(file_path)
count = 1

for file_name in fall_list:
    # 파일 확장자가 .wav인 경우에만 처리
    if file_name.endswith('.wav'):
        full_file_name = os.path.join(file_path, file_name)
        output_file_name = os.path.join(output_path, f'NF{count}.csv')
        
        y, sr = librosa.load(full_file_name)
        duration = librosa.get_duration(y=y)
        onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=2205, aggregate=np.median)
        
        pd.set_option('display.max_columns', None)
        df = pd.DataFrame(onset_env)
        df.to_csv(output_file_name)
        
        count = count + 1

#### Combining Data

In [4]:
# %pip install audiomentations

In [5]:
# Function to read the second column from CSV files in a directory
def read_second_column_csv(directory):
    dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
            flattened_df = df.T
            dataframes.append(flattened_df)
    return pd.concat(dataframes, ignore_index=True)

# Function to apply various augmentations to audio data
def augment_audio_data(y, sr, augmentations=['noise', 'shift', 'stretch', 'pitch']):
    augmented_data = []
    for aug in augmentations:
        if aug == 'noise':
            noise_amp = 0.005 * np.random.uniform() * np.amax(y)
            y_aug = y + noise_amp * np.random.normal(size=y.shape[0])
        elif aug == 'shift':
            shift_range = int(np.random.uniform(low=-5, high=5) * sr)
            y_aug = np.roll(y, shift_range)
        elif aug == 'stretch':
            rate = np.random.uniform(low=0.8, high=1.2)
            y_aug = librosa.effects.time_stretch(y, rate=rate)
        elif aug == 'pitch':
            n_steps = np.random.uniform(low=-1, high=1)
            y_aug = librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)
        augmented_data.append(y_aug)
    return augmented_data


# Function to add augmentations to a dataset
def add_augmentations(directory, sr=16000, augmentations=['noise', 'shift', 'stretch', 'pitch'], repeats=5):
    augmented_dataframes = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, filename), header=None, usecols=[1])
            y = df.iloc[:, 0].values
            for _ in range(repeats):  # Repeat the augmentations
                augmented_clips = augment_audio_data(y, sr, augmentations)
                for aug_clip in augmented_clips:
                    aug_features = librosa.feature.mfcc(y=aug_clip, sr=sr, n_mfcc=13)
                    flattened_features = np.mean(aug_features, axis=1).reshape(1, -1)
                    augmented_dataframes.append(pd.DataFrame(flattened_features))
    return pd.concat(augmented_dataframes, ignore_index=True)


def detect_outliers_iqr(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    outliers = (data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))
    return outliers

In [6]:
from sklearn.utils import resample
from imblearn.over_sampling import KMeansSMOTE

In [7]:
fall_dir = 'Mono_Audio_Feature/Fall_Feature'
non_fall_dir = 'Mono_Audio_Feature/Non_Fall_Feature/'

In [8]:
# Read and combine CSV files
falls_data = read_second_column_csv(fall_dir)
not_falls_data = read_second_column_csv(non_fall_dir)

# Apply augmentations
falls_augmented_data = add_augmentations(fall_dir)
not_falls_augmented_data = add_augmentations(non_fall_dir)

# Combine original and augmented data
combined_data = pd.concat([falls_data, not_falls_data, falls_augmented_data, not_falls_augmented_data])
combined_data['label'] = [1] * len(falls_data) + [0] * len(not_falls_data) + \
                         [1] * len(falls_augmented_data) + [0] * len(not_falls_augmented_data)

outliers = detect_outliers_iqr(combined_data)
combined_data[outliers] = np.nan

# Apply K-Means SMOTE for balancing
kmeans_smote = KMeansSMOTE(random_state=123, k_neighbors=10, cluster_balance_threshold=0.1)
X = combined_data.drop('label', axis=1)
y = combined_data['label']
X = X.fillna(0)
X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

# Combine resampled features and labels into a DataFrame
resampled_data = pd.DataFrame(X_resampled, columns=X.columns)
resampled_data['label'] = y_resampled

# Shuffle the dataset
resampled_data = resampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Separate the dataset by label
falls_data_label = resampled_data[resampled_data['label'] == 1]
not_falls_data_label = resampled_data[resampled_data['label'] == 0]

# # Sample 350 entries from each subset
# sampled_falls_data = falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)
# sampled_not_falls_data = not_falls_data_label.sample(n=1200, random_state=123).reset_index(drop=True)

# Combine the sampled data
combined_sampled_data = pd.concat([falls_data_label, not_falls_data_label]).reset_index(drop=True)

# Shuffle the combined sampled dataset
final_sampled_data = combined_sampled_data.sample(frac=1, random_state=123).reset_index(drop=True)

# Save to CSV
final_sampled_data.to_csv('final_dataset.csv', index=False)

c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=24
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=29
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=30
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=22
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-packages\librosa\core\spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=26
  warnings.warn(
c:\Users\wonha\AppData\Local\Programs\Python\Python39\lib\site-package

In [9]:
final_sampled_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,label
0,209.848960,30.175270,-1.544584,0.822148,19.195339,-11.920886,-3.227149,7.202528,-4.136320,-0.239372,3.606225,-4.276194,2.390917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,190.904797,25.567855,18.495206,6.886968,-2.620626,-3.872560,4.502386,-5.092250,5.412058,-3.717602,2.455735,-3.438875,5.028288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,264.625285,5.167377,8.794453,7.398150,8.345742,8.764613,-5.701966,-5.577624,5.394306,-2.646017,-0.964748,2.571240,0.061898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,265.729573,12.975963,2.853907,-7.288065,6.884959,-4.403016,2.221964,-0.862579,0.180973,0.072143,-0.096388,0.008779,0.104212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,154.601014,38.841208,27.287872,-4.507218,-7.488972,8.998490,1.679976,-9.192562,4.574674,2.519085,-2.144742,-1.931476,1.529405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438,210.611454,41.492095,-19.536418,5.810041,1.198346,-3.598911,6.709896,-2.508441,4.740065,-3.402765,0.742813,-0.105644,1.109151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0